In [1]:
from src.model.live_test import YahooFinanceTrader

In [2]:
livetest = YahooFinanceTrader(None, symbol="SPY", threshold_upper=0.5, threshold_lower=-0.5)

In [3]:
livetest.fetch_initial_data()

Successfully fetched initial 100 bars for SPY


True

In [4]:
livetest.data_buffer

,timestamp,open,high,low,close,volume,Dividends,Stock Splits,Capital Gains
290,2025-03-13 14:20:00-04:00,552.020020,552.460022,551.880005,552.460022,95895,0.0,0.0,0.0
291,2025-03-13 14:21:00-04:00,552.469971,552.520020,552.130188,552.370117,88732,0.0,0.0,0.0
292,2025-03-13 14:22:00-04:00,552.409973,552.440002,552.289978,552.309998,35602,0.0,0.0,0.0
293,2025-03-13 14:23:00-04:00,552.309998,552.580017,552.169983,552.570007,67093,0.0,0.0,0.0
294,2025-03-13 14:24:00-04:00,552.549988,552.799988,552.489990,552.640015,89039,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
385,2025-03-13 15:55:00-04:00,550.909973,551.299988,550.869995,550.969971,1486649,0.0,0.0,0.0
386,2025-03-13 15:56:00-04:00,550.969971,551.349976,550.919983,551.140015,327823,0.0,0.0,0.0
387,2025-03-13 15:57:00-04:00,551.140015,551.229980,550.909973,550.989990,450305,0.0,0.0,0.0
388,2025-03-13 15:58:00-04:00,550.989990,551.090027,550.820007,550.859985,709997,0.0,0.0,0.0
